# POD-NN method

## The advection-diffusion problem

Let us  considere the following BVP. It is based on the 2D stationary advection-diffusion. It is here parametrized by the diffusivity coefficient $\lambda(\mu)$. The equations read: 

$$\begin{equation}
\begin{cases}
-div(\lambda(\mu)\nabla u)+w\nabla u=f & in~~\Omega\\
~~~~~~\hfill~~~~~~~~~~~~~~~~~~u=g & in~~\varGamma_{in}\\
~~~\hfill~~~~-\lambda(\mu)\nabla u.n=0 & in~~\varGamma_{wall}\\
~~~\hfill~~~~-\lambda(\mu)\nabla u.n=0 & in~~\varGamma_{out}
\end{cases}
\label{eq:Advection-non-affine}
\end{equation}$$

with $\lambda(\mu)=\exp(\mu_{0}(\mu + 1))$. The scalar $\mu_{0}=0.7$ and $\mu\in\textbf{P}=[\mu_{min},{~}\mu_{max}],{~}  \mu_{min}=1,{~} \mu_{max}=10$.

### Modules  import and initializations

In [ ]:
### Modules importation
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.datasets import mnist
import tensorflow.keras.preprocessing.image as kpi
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kloss
import tensorflow.keras.regularizers as kr
import tensorflow.keras.backend as K
import tensorflow.keras.utils as ku
from tensorflow.keras import callbacks
import scipy as sc
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
import sys,os
import random 
#import pandas as pd
import sklearn.utils
# Verbosity
fit_verbosity = 1

### Normalization and non-affine functions 

In [2]:
# The scaling function for data normalisation
# Scaling function
def scaling(S, S_max, S_min):
    S[ : ] = (S - S_min)/(S_max - S_min)
    
# Inverse scaling function
def inverse_scaling(S, S_max, S_min):
    S[ : ] = (S_max - S_min) * S + S_min
    
# The non-affinity function
def Lambda(mu):
    return np.exp(mu0*(mu+1.))

## The offline phase

### Preparing the data

The RB POD matrix $Brb=[\xi_{1},\dots,\xi_{M}]\in\mathbb{R}^{NN\times N_{rb}}$. The RB matrix is constructed by POD method. It is computed for 100 snapshots of Problem (1). The reduced matrix here is of size $N_{rb}=5$. $$\$$
For the training step of the NN, the snapshots matrix $S=[U_{h}(\mu_{1}),\dots,U_{h}(\mu_{M})]\in\mathbb{R}^{NN\times M}$ has already been computed with $M=10000$.

#### Load the data from numpy files

In [3]:
# Prepare the data
# Load the data from the numpy file
# Snaphots matrix
S =  np.load('Snapshots_non_affine.npy') # of size M*NN
S = S.transpose() # of size NN*M

# The reduced POD basis
Brb = np.load('Brb.npy') # of size NN*Nrb#
Brb = Brb.real
Nrb = len(Brb.transpose())

# The parameter matrix
P = np.load('parameter_non_affine.npy') # of size M x 1
P = P.reshape(len(P),1)

#### Computation of the reduced outputs for the NN and randomly shuffle the data

The reduced outputs are computed by the formula: $$\begin{equation}Urb = Brb^{T}U_{h}(\mu)\in\mathbb{R}^{N_{rb}}\end{equation}$$ with $\mu\in\textbf{P}=[\mu_{min},{~}\mu_{max}]$. $\\$
We denote by $Urb_{POD}=Brb^{T}S$.

In [ ]:
# Computation of the reduced solutions: Brb^T*Uh(mu)
Urb_POD = np.dot(Brb.transpose(),S) # of size Nrb*M
#print("Urb_POD size=",Urb_POD.shape)

# Transpose 
Urb_POD = Urb_POD.transpose() # of size M*Nrb
print("Urb_POD size=",Urb_POD.shape)

# Randomy shufl the data set
shuffle = np.arange(len(Urb_POD))
np.random.shuffle(shuffle)
Urb_POD = Urb_POD[shuffle]
P = P[shuffle]
print("Urb_POD before normalization",Urb_POD)
print("P before normalization",P)

#### Data normalization

The normalization of the data is done as follows: $$\\$$
The input parameters for the NN are such that: $$\begin{equation}\tilde{\mu_{i}} = \frac{\mu_{i}}{\mu_{max}}\end{equation}$$ for $1\leq i\leq M$ $$\$$
The outputs (RB solutions) for the NN are normalized as follows:$$\begin{equation}(\tilde{Urb}_{POD})_{ij} = \frac{(Urb_{POD})_{ij}-\underset{i,j}{\min}(Urb_{POD})_{ij}}{\underset{i,j}{\max}(Urb_{POD})_{ij}-\underset{i,j}{\min}(Urb_{POD})_{ij}}\end{equation}$$ for $1\leq i\leq M$ and $1\leq j\leq N_{rb}$ $$\\$$

In [ ]:
# Data normalization
# Normalization of the parameter set
# Obtain the min and the max of P
P_max = np.max(P); P_min = np.min(P)
# Normalize the parameter vector P
P = P/P_max

# Normalization of the reduced matrix
# Obtain the min and the max of the reduced outputs BrbUh
Urb_POD_max = np.max(Urb_POD); Urb_POD_min = np.min(Urb_POD)
scaling(Urb_POD, Urb_POD_max, Urb_POD_min)

print("P after normalization",P)
print("Urb_POD after normalization",Urb_POD)

#### Load the already trained NN

In [ ]:
# Load the pre-trained NN stored in h5 format
Model = km.load_model('Neural-network.h5', custom_objects={'mse': tf.keras.losses.MeanSquaredError()})

# Summary of the model: layers and number of parameters 
Model.summary()

## The online phase

### New parameter value

In [ ]:
# The onlinbe phase 
# Predict the new solution U_POD_NN
# The constant mu_0
mu0 = 0.7

# New value of the physical parameter mu
print('New value for mu ')
# The parameter input for the NN
mu = np.array([[float(input())]])
print("mu=",mu)

# The non-affine parameter
# TO BE COMPLETED ...
diffus = Lambda(mu)
print("diffus=",diffus)

# Normalization of the non-affine parameter
# TO BE COMPLETED ...
diffus = diffus/P_max
print("diffus after normalization=",diffus)

### Compute the RB solution for the new parameter value by performing the NN

In [ ]:
# Predict the reduced basis solution of the new parameter
# TO BE COMPLETED ...
U_POD_NN = Model.predict(diffus)
print("U_POD_NN=",U_POD_NN)

# Rescaling the predicted reduced basis solution
# TO BE COMPLETED ...
inverse_scaling(U_POD_NN, Urb_POD_max, Urb_POD_min)
print("U_POD_NN after rescaling=",U_POD_NN)

# The change of variable from the RB basis to complete FE one.
# TO BE COMPLETED ...
U_FE_NN = np.dot(Brb,U_POD_NN.transpose())
print("U_FE_NN=",U_FE_NN)

### Save the POD-NN solution in numpy file

In [ ]:
np.save('Uh_POD_NN',Uh_POD_NN)